# Convert text to a graph

This notebook demonstrates how to extract graph from any text using the graph maker

Steps:
1. Split the document
2. Construct the knowledge graph
3. Save the graph to a graph database

In [1]:
import os

In [2]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = "TEST"

In [3]:
from knowledge_graph_maker import GraphMaker, Ontology, OpenAIClient
from knowledge_graph_maker import Document

## Split the document

In [418]:
#from langchain_experimental.text_splitter import SemanticChunker
# from langchain_openai.embeddings import OpenAIEmbeddings

In [419]:
# text_splitter = SemanticChunker(OpenAIEmbeddings())
# docs1 = text_splitter.split_text(temp)

In [4]:
from langchain import hub
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from typing import Optional, List

### Extract propositions from each paragraph

In this part, we will use an LLM to extract stand alone statements from a raw piece of text.

Example: Greg went to the park. He likes walking > ['Greg went to the park.', 'Greg likes walking']

Pulling out propositions is done via a well-crafted prompt. I'm going to pull it from LangHub, LangChain's home for prompts.

Use custom prompt to instruct an LLM to extract propositions.

In [5]:
obj = hub.pull("wfh/proposal-indexing")

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


We will be using gpt-4o-mini to extract the propositions

In [6]:
chunking_llm = ChatOpenAI(model='gpt-4o-mini', openai_api_key = os.getenv("OPENAI_API_KEY", 'YouKey'))

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Then I'll make a runnable w/ langchain, this'll be a short way to combine the prompt and llm

In [7]:
# use it in a runnable
runnable = obj | chunking_llm

The output from a runnable is a json-esque structure in a string. We need to pull the sentences out. I found that LangChain's example extraction was giving me a hard time so I'm doing it manually with a pydantic data class. There is definitely room to improve this.

Create your class then put it in an extraction chain.

In [8]:
# Pydantic data class
class Sentences(BaseModel):
    sentences: List[str]
    
# Extraction
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=chunking_llm)

/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: LangChain has introduced a method called `with_structured_output` thatis available on ChatModels capable of tool calling.You can read more about the method here: <https://python.langchain.com/docs/modules/model_io/chat/structured_output/>. Please follow our extraction use case documentation for more guidelineson how to do information extraction with LLMs.<https://python.langchain.com/docs/use_cases/extraction/>. If you notice other issues, please provide feedback here:<https://github.com/langchain-ai/langchain/discussions/18154>
  warn_deprecated(


Define a function to extract propositions

In [9]:
def get_propositions(text):
    runnable_output = runnable.invoke({
    	"input": text
    }).content
    
    propositions = extraction_chain.run(runnable_output)[0].sentences
    return propositions

Load source document

In [10]:
with open('./data_input/eldenring.txt') as file:
    essay = file.read()

Then you need to decide what you send to your proposal maker. The prompt has an example that is about 1K characters long. So I would experiment with what works for you. This isn't another chunking decision, just pick something reasonable and try it out.

I'm using paragraphs

Split the documents at each line break

In [11]:
paragraphs = essay.split("\n\n")

Let's see how many we have

In [12]:
len(paragraphs)

66

In [13]:
paragraphs

['Lore in Elden Ring covers all the information related to the world and mythos of the game. From past events that occurred long ago, to the history of the Lands Between and the mystery of the elusive Elden Ring, all can be found here. Please note that this page contains heavy spoilers.',
 '- The Mythos of Elden Ring was written by George R. R. Martin. Mythos refers to the overall narrative theme or plot structure. It can also be interpreted as a belief system.\n- The Story of Elden Ring was written by Hidetaka Miyazaki and his team at FromSoftware\n- The Lore and Story of Elden Ring are told in a similar manner to other FromSofware "Souls" games, so players should expect to find plenty left to interpretation as well as seemingly contradictory statements about various game characters, elements, and concepts.',
 'Elden Ring Lore Overview',
 'The storyteller folds her slender hands - both pairs - and speaks. “It happened an age ago. But when I recall, I see it true.” So begins the tale o

For each paragraph, extract the propositions then put it in a list of propositions

In [15]:
parag = paragraphs[0]
propo = get_propositions(parag)


Lore in Elden Ring covers all the information related to the world and mythos of the game. From past events that occurred long ago, to the history of the Lands Between and the mystery of the elusive Elden Ring, all can be found here. Please note that this page contains heavy spoilers.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


['Lore in Elden Ring covers all the information related to the world and mythos of the game.', 'The information covered by the lore includes past events that occurred long ago.', 'The information covered by the lore includes the history of the Lands Between.', 'The information covered by the lore includes the mystery of the elusive Elden Ring.', 'All information related to the world and mythos of Elden Ring can be found in this lore.', 'This page contains heavy spoilers.']


/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)


In [17]:
parag

'Lore in Elden Ring covers all the information related to the world and mythos of the game. From past events that occurred long ago, to the history of the Lands Between and the mystery of the elusive Elden Ring, all can be found here. Please note that this page contains heavy spoilers.'

In [16]:
propo

['Lore in Elden Ring covers all the information related to the world and mythos of the game.',
 'The information covered by the lore includes past events that occurred long ago.',
 'The information covered by the lore includes the history of the Lands Between.',
 'The information covered by the lore includes the mystery of the elusive Elden Ring.',
 'All information related to the world and mythos of Elden Ring can be found in this lore.',
 'This page contains heavy spoilers.']

In [18]:
parag1 = paragraphs[1]
print(parag1)

- The Mythos of Elden Ring was written by George R. R. Martin. Mythos refers to the overall narrative theme or plot structure. It can also be interpreted as a belief system.
- The Story of Elden Ring was written by Hidetaka Miyazaki and his team at FromSoftware
- The Lore and Story of Elden Ring are told in a similar manner to other FromSofware "Souls" games, so players should expect to find plenty left to interpretation as well as seemingly contradictory statements about various game characters, elements, and concepts.


In [21]:
propo1 = get_propositions(parag1)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

In [22]:
propo1

['The Mythos of Elden Ring was written by George R. R. Martin.']

In [23]:
parag2 = 'The Mythos of Elden Ring was written by George R. R. Martin. Mythos refers to the overall narrative theme or plot structure. It can also be interpreted as a belief system. The Story of Elden Ring was written by Hidetaka Miyazaki and his team at FromSoftware. The Lore and Story of Elden Ring are told in a similar manner to other FromSofware "Souls" games, so players should expect to find plenty left to interpretation as well as seemingly contradictory statements about various game characters, elements, and concepts'

propo2 = get_propositions(parag2)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

In [24]:
propo2

['The Mythos of Elden Ring was written by George R. R. Martin.']

In [25]:
parag2 = paragraphs[2]
prop2 = get_propositions(parag2)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

In [29]:
paragraphs[3]

'The storyteller folds her slender hands - both pairs - and speaks. “It happened an age ago. But when I recall, I see it true.” So begins the tale of the Shattering, a devastating war between the children of Marika, Demigods of the Lands Between. One grim night in the depths of winter, a flock of unknown assassins stole across the Lands Between.\nIn a coetaneous attack, this foul covenant snuffed out the lives of many of the God-Queen’s kin throughout the empire, too numerous and too scattered for her godly protection to save. The assassins’ targets were multifold, but none was as devastating a loss to the Eternal Queen as that of Godwyn the Golden. After his death, the Elden Ring was somehow shattered, and the order of the world broke with it.\nOut of this chaos came war. Open, bitter war, pitting blood against blood and kin against kin. The conflict broke a vast and prosperous empire into brittle fragments. No single side could find enough purchase to maintain a decisive victory, eve

In [26]:
print(parag2)
print(prop2)

Elden Ring Lore Overview
['Elden Ring is a video game.', 'Elden Ring has a lore overview.']


In [ ]:
proposition_list = []

for para in paragraphs:
    propositions = get_propositions(para)
    
    proposition_list.extend(propositions)
    # print (f"Done with {i}")

Let's take a look at what the propositions look like

In [15]:
print (f"You have {len(proposition_list)} propositions")
proposition_list

You have 583 propositions


['Lore in Elden Ring covers all the information related to the world of Elden Ring.',
 'Lore in Elden Ring covers all the information related to the mythos of Elden Ring.',
 'Lore in Elden Ring includes past events that occurred long ago.',
 'Lore in Elden Ring includes the history of the Lands Between.',
 'Lore in Elden Ring includes the mystery of the elusive Elden Ring.',
 'All information related to the world and mythos of Elden Ring can be found on this page.',
 'This page contains heavy spoilers.',
 'The Mythos of Elden Ring was written by George R. R. Martin.',
 'The Shattering is a devastating war between the children of Marika.',
 'The children of Marika are Demigods of the Lands Between.',
 "After Godwyn the Golden's death, the Elden Ring was somehow shattered.",
 'The power of a Great Rune fueled the conflict.',
 'The battle of General Radahn against Malenia the Severed illustrated that neither strength nor skill alone would be enough to take control of the entirety of the l

In [16]:
from proposition2 import eldenring_propositions as erp

In [17]:
print (f"You have {len(erp)} propositions")

You have 583 propositions


In [20]:
erp
proposition_list = erp[:11]

proposition_list

['Lore in Elden Ring covers all the information related to the world of Elden Ring.',
 'Lore in Elden Ring covers all the information related to the mythos of Elden Ring.',
 'Lore in Elden Ring includes past events that occurred long ago.',
 'Lore in Elden Ring includes the history of the Lands Between.',
 'Lore in Elden Ring includes the mystery of the elusive Elden Ring.',
 'All information related to the world and mythos of Elden Ring can be found on this page.',
 'This page contains heavy spoilers.',
 'The Mythos of Elden Ring was written by George R. R. Martin.',
 'The Shattering is a devastating war between the children of Marika.',
 'The children of Marika are Demigods of the Lands Between.',
 "After Godwyn the Golden's death, the Elden Ring was somehow shattered."]

So you'll see that they look like regular sentences, but they are actually statements that are able to stand on their own. For example, one of the sentences in the raw text is "They meant well, but this is rarely true." if you were to chunk that on it's own, the LLM would have no idea who you're talking about. Who meant well? What is rarely true? But those have been covered by the propositions.

### Group each proposition to a chunk (Optional)

 Use an LLM that can reason about each proposition and determine whether or not it should be a part of an existing chunk or if a new chunk should be made.


 (Note: After testing, i found out that grouping each proposition to a chunks is not every effective.)

In [25]:
from agentic_chunker import AgenticChunker

In [26]:
ac = AgenticChunker()

In [27]:
ac.add_propositions(erp)


Adding: 'Lore in Elden Ring covers all the information related to the world of Elden Ring.'
No chunks, creating a new one


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

Created new chunk (36362): Elden Ring Lore & World-Building

Adding: 'Lore in Elden Ring covers all the information related to the mythos of Elden Ring.'


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

Chunk Found (36362), adding to: Elden Ring Lore & World-Building


...

Chunk Found (9d5c2), adding to: Nepheli Loux: Warrior, Justice, and Legacy


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan


Adding: 'Kenneth Haight is the owner of Fort Haight.'


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/Users/user/miniconda3/envs/graphmaker/lib/python3.11/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=Pydan

TypeError: object of type 'NoneType' has no len()

In [28]:
ac.pretty_print_chunks()


You have 12 chunks

Chunk #0
Chunk ID: 36362
Summary: This chunk contains detailed information about the lore, mythos, and history of the Elden Ring universe, focusing on the shattering of the Elden Ring, the roles of key characters, and the dynamics of the Lands Between; please add any new propositions that explore the significance of the Great Runes, the nature of the demigods' powers, and the overarching themes of power, grace, and the influence of the Frenzied Flame, as well as further details about character backgrounds, motivations, and their interactions within the narrative.
Propositions:
    -Lore in Elden Ring covers all the information related to the world of Elden Ring.
    -Lore in Elden Ring covers all the information related to the mythos of Elden Ring.
    -Lore in Elden Ring includes past events that occurred long ago.
    -Lore in Elden Ring includes the history of the Lands Between.
    -Lore in Elden Ring includes the mystery of the elusive Elden Ring.
    -All inf

In [29]:
ac.pretty_print_chunk_outline()

Chunk Outline

Chunk ID: 36362
Chunk Name: Elden Ring Lore & History
Chunk Summary: This chunk contains detailed information about the lore, mythos, and history of the Elden Ring universe, focusing on the shattering of the Elden Ring, the roles of key characters, and the dynamics of the Lands Between; please add any new propositions that explore the significance of the Great Runes, the nature of the demigods' powers, and the overarching themes of power, grace, and the influence of the Frenzied Flame, as well as further details about character backgrounds, motivations, and their interactions within the narrative.

Chunk ID: 20407
Chunk Name: Spoilers
Chunk Summary: This chunk contains information about content that includes significant spoilers.

Chunk ID: a12a9
Chunk Name: Geographical Locations
Chunk Summary: This chunk contains information about geographical locations and areas.

Chunk ID: 9a920
Chunk Name: Video Games
Chunk Summary: This chunk contains information about video games,

In [30]:
print(ac.get_chunks(get_type='list_of_strings'))

["Lore in Elden Ring covers all the information related to the world of Elden Ring. Lore in Elden Ring covers all the information related to the mythos of Elden Ring. Lore in Elden Ring includes past events that occurred long ago. Lore in Elden Ring includes the history of the Lands Between. Lore in Elden Ring includes the mystery of the elusive Elden Ring. All information related to the world and mythos of Elden Ring can be found on this page. The Mythos of Elden Ring was written by George R. R. Martin. The Shattering is a devastating war between the children of Marika. The children of Marika are Demigods of the Lands Between. After Godwyn the Golden's death, the Elden Ring was somehow shattered. The power of a Great Rune fueled the conflict. The battle of General Radahn against Malenia the Severed illustrated that neither strength nor skill alone would be enough to take control of the entirety of the lands Marika had united. The Shattering was initiated by the demigods’ hunger for po

In [31]:
mylist = ac.get_chunks(get_type='list_of_strings')

In [32]:
len(mylist)

12

In [33]:
mylist

["Lore in Elden Ring covers all the information related to the world of Elden Ring. Lore in Elden Ring covers all the information related to the mythos of Elden Ring. Lore in Elden Ring includes past events that occurred long ago. Lore in Elden Ring includes the history of the Lands Between. Lore in Elden Ring includes the mystery of the elusive Elden Ring. All information related to the world and mythos of Elden Ring can be found on this page. The Mythos of Elden Ring was written by George R. R. Martin. The Shattering is a devastating war between the children of Marika. The children of Marika are Demigods of the Lands Between. After Godwyn the Golden's death, the Elden Ring was somehow shattered. The power of a Great Rune fueled the conflict. The battle of General Radahn against Malenia the Severed illustrated that neither strength nor skill alone would be enough to take control of the entirety of the lands Marika had united. The Shattering was initiated by the demigods’ hunger for po

## Create the knowledge graph

### Define the Ontology

The ontology is a pydantic model with the following schema. 

```python
class Ontology(BaseModel):
    label: List[Union[str, Dict]]
    relationships: List[str]
```



In [21]:
ontology = Ontology(
    labels=[
        {"Person": "Person name without any adjectives, Remember a person may be referenced by their name or using a pronoun"},
        {"Object": "Do not add the definite article 'the' in the object name"},
        {"Event": "Event involving multiple people. Do not include qualifiers or verbs like gives, leaves, works etc."},
        {"Place": "Locations where specific events took place"},
        {"Faction": "Name of a group that a person belongs without any adjectives, Remember, a person may one belong to one faction"},
        {"Miscellaneous": "Any important concept can not be categorised with any other given label"},
    ],
    relationships=[
        "Relation between any pair of Entities"
        ],
)

### Select a LLM 

This model will:
- Generate summaries each proposition
- Extract the nodes and edges from the pydantic documents


In [22]:
## Groq models
model = "mixtral-8x7b-32768"
# model ="llama3-8b-8192"
# model = "llama3-70b-8192"
# model="gemma-7b-it"

## Open AI models
oai_model="gpt-4o-mini"

## Use Groq
# llm = GroqClient(model=model, temperature=0.1, top_p=0.5)

## OR Use OpenAI
llm = OpenAIClient(model=oai_model, temperature=0.1, top_p=0.5)

### Create pydantic documents from chunks

Documents is a pydantic model with the following schema 

```python
class Document(BaseModel):
    text: str
    metadata: dict
```

The metadata we add to the document here is copied to every relation that is extracted out of the document. More often than not, the node pairs have multiple relation with each other. The metadata helps add more context to these relations

In this example I am generating a summary of the text chunk, and the timestamp of the run, to be used as metadata. 

In [23]:
import datetime
current_time = str(datetime.datetime.now())

In [24]:
def generate_summary(text):
    SYS_PROMPT = (
        "Succintly summarise the text provided by the user. "
        "Respond only with the summary and no other comments"
    )
    try:
        summary = llm.generate(user_message=text, system_message=SYS_PROMPT)
    except:
        summary = ""
    finally:
        return summary

In [33]:
docs = map(
    lambda t: Document(text=t, metadata={"summary": generate_summary(t), 'generated_at': current_time}),
    proposition_list
)

In [34]:
mylist = list(docs)
mylist

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


...

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


[Document(text='Lore in Elden Ring covers all the information related to the world of Elden Ring.', metadata={'summary': 'Lore in Elden Ring encompasses all information pertaining to its world.', 'generated_at': '2024-08-30 11:50:13.178102'}),
 Document(text='Lore in Elden Ring covers all the information related to the mythos of Elden Ring.', metadata={'summary': 'Lore in Elden Ring encompasses the mythos and background information of the game.', 'generated_at': '2024-08-30 11:50:13.178102'}),
 Document(text='Lore in Elden Ring includes past events that occurred long ago.', metadata={'summary': "Elden Ring's lore encompasses historical events from the distant past.", 'generated_at': '2024-08-30 11:50:13.178102'}),
 Document(text='Lore in Elden Ring includes the history of the Lands Between.', metadata={'summary': "Elden Ring's lore encompasses the history and background of the Lands Between.", 'generated_at': '2024-08-30 11:50:13.178102'}),
 Document(text='Lore in Elden Ring includes t

### Generate graph from pydantic documents

In [26]:
graph_maker = GraphMaker(ontology=ontology, llm_client=llm, verbose=False)

In [27]:
graph = graph_maker.from_documents(
    list(docs), 
    delay_s_between=0 ## delay_s_between because otherwise groq api maxes out pretty fast. 
    ) 
print("Total number of Edges", len(graph))

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Using Model:  gpt-4o-mini


...

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-30 11:50:55 - INFO 
LLM Response:
[
   {
       "node_1": {"label": "Person", "name": "Marika"},
       "node_2": {"label": "Miscellaneous", "name": "Demigods of the Lands Between"},
       "relationship": "Marika is the parent of the Demigods of the Lands Between."
   }
]


▶︎ GRAPH MAKER LOG - 2024-08-30 11:50:55 - INFO 
Trying JSON Parsing: 
[
   {
       "node_1": {"label": "Person", "name": "Marika"},
       "node_2": {"label": "Miscellaneous", "name": "Demigods of the Lands Between"},
       "relationship": "Marika is the parent of the Demigods of the Lands Between."
   }
]


▶︎ GRAPH MAKER LOG - 2024-08-30 11:50:55 - INFO 
JSON Parsing Successful!


▶︎ GRAPH MAKER LOG - 2024-08-30 11:50:55 - INFO 
Document: 11


▶︎ GRAPH MAKER VERBOSE - 2024-08-30 11:50:55 - INFO 
Using Ontology:
labels=[{'Person': 'Person name without any adjectives, Remember a person may

Using Model:  gpt-4o-mini


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

▶︎ GRAPH MAKER VERBOSE - 2024-08-30 11:50:57 - INFO 
LLM Response:
[
    {
        "node_1": {
            "label": "Person",
            "name": "Godwyn"
        },
        "node_2": {
            "label": "Object",
            "name": "Elden Ring"
        },
        "relationship": "Godwyn's death is associated with the shattering of the Elden Ring."
    }
]


▶︎ GRAPH MAKER LOG - 2024-08-30 11:50:57 - INFO 
Trying JSON Parsing: 
[
    {
        "node_1": {
            "label": "Person",
            "name": "Godwyn"
        },
        "node_2": {
            "label": "Object",
            "name": "Elden Ring"
        },
        "relationship": "Godwyn's death is associated with the shattering of the Elden Ring."
    }
]


▶︎ GRAPH MAKER LOG - 2024-08-30 11:50:57 - INFO 
JSON Parsing Successful!



Total number of Edges 12


In [35]:
graph

[Edge(node_1=Node(label='Miscellaneous', name='Lore'), node_2=Node(label='Place', name='Elden Ring'), relationship='Lore contains information related to the world of Elden Ring.', metadata={'summary': 'Lore in Elden Ring encompasses all information about its world.', 'generated_at': '2024-08-30 11:50:13.178102'}, order=0),
 Edge(node_1=Node(label='Miscellaneous', name='Lore'), node_2=Node(label='Miscellaneous', name='Elden Ring'), relationship='Lore contains information related to the mythos of Elden Ring.', metadata={'summary': "Lore in Elden Ring encompasses the mythos and background information of the game's universe.", 'generated_at': '2024-08-30 11:50:13.178102'}, order=1),
 Edge(node_1=Node(label='Miscellaneous', name='Lore'), node_2=Node(label='Event', name='past events'), relationship='Lore includes past events that occurred long ago.', metadata={'summary': "Elden Ring's lore encompasses historical events from the distant past.", 'generated_at': '2024-08-30 11:50:13.178102'}, o

In [36]:
for edge in graph:
    print(edge.model_dump)
    print(edge.model_dump(exclude=['metadata']), "\n\n")

<bound method BaseModel.model_dump of Edge(node_1=Node(label='Miscellaneous', name='Lore'), node_2=Node(label='Place', name='Elden Ring'), relationship='Lore contains information related to the world of Elden Ring.', metadata={'summary': 'Lore in Elden Ring encompasses all information about its world.', 'generated_at': '2024-08-30 11:50:13.178102'}, order=0)>
{'node_1': {'label': 'Miscellaneous', 'name': 'Lore'}, 'node_2': {'label': 'Place', 'name': 'Elden Ring'}, 'relationship': 'Lore contains information related to the world of Elden Ring.', 'order': 0} 


<bound method BaseModel.model_dump of Edge(node_1=Node(label='Miscellaneous', name='Lore'), node_2=Node(label='Miscellaneous', name='Elden Ring'), relationship='Lore contains information related to the mythos of Elden Ring.', metadata={'summary': "Lore in Elden Ring encompasses the mythos and background information of the game's universe.", 'generated_at': '2024-08-30 11:50:13.178102'}, order=1)>
{'node_1': {'label': 'Miscellaneous

## Save the graph to a graph database

In [37]:
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "12345678"
os.environ["NEO4J_URI"]= "bolt://localhost:7687"

In [38]:
from knowledge_graph_maker import Neo4jGraphModel

create_indices = False
neo4j_graph = Neo4jGraphModel(edges=graph, create_indices=create_indices)
neo4j_graph.save()

12